In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [9]:
# Redefine path when not using Nansen
df = pd.read_csv('/Users/david.hedges/projects/shodair/gsa_uncommented.vcf', sep='\t')
df_labels = pd.read_excel('/Users/david.hedges/projects/shodair/regDocs/Copy of PGX risperidone only Erica7_23_2019.xlsx')

In [10]:
# Encode data as 0,1,2 and clear out non-genetic data
df = df.fillna(0)
df = df.replace('0/0',0)
df = df.replace('0/1',1)
df = df.replace('1/1',2)
df = df.replace('./.',0)
df = df.drop(columns=['#CHROM','POS','REF','ALT','QUAL','FILTER','INFO','FORMAT'])

In [11]:
# Rename Headers in VCF to only have Shodair ID
header = list(df)
vals = []
for i in header:
    try:
        x = re.search(r'SCH-(\d+)', i).group(0)
        vals.append(x)
    except:
        vals.append(i)

column_dict = dict(zip(header,vals))
df = df.rename(index=str, columns=column_dict)


# Reset Index to be Shodair ID and transpose dataframe
df = df.set_index('ID')
df_T = df.T
df_T.shape


# Rename headers in labels to be identical to headers in VCF
ID = ['SCH-'+str(i) for i in df_labels['Random ID']]
df_labels['ID'] = ID

# Clean out unnecessary headers
df_labels = df_labels.drop(columns=['Random ID','Plate','Sample Order','Plate ID Location','Gender','Concentration',
                         '260/2030 via nanodrop','Enzyme','Gene1','Gene2','Phenotype','Race','COMMENT',
                         'Allele Activity Score 1','Allele Activity Score 2','Diplotype Activity Score',
                         'Metabollizer Status'])
df_labels = df_labels.set_index('ID')
df_labels

# Combine dataframes to get drug info in dataframe 
df_merged = df_T.join(df_labels)
df_merged

,GSA-rs114420996,rs10458597,ilmnseq_rs9701296,rs9701055,GSA-rs9283150,GSA-rs9326622,ilmnseq_rs9651229_F2BT,ilmnseq_rs9701872,ilmnseq_rs9701872_ilmndup1,ilmnseq_rs11497407,...,chrM_16465,chrM_16465_ilmndup1,200610-37,rs3937033,ilmnseq_rs386829315,ilmnseq_rs386829316,Aripiprazole,Olanzapine,Quetiapine,Risperidone
SCH-845,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,1,1,1,1
SCH-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
SCH-463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
SCH-253,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
SCH-522,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
SCH-794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
SCH-187,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
SCH-615,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
SCH-911,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
SCH-807,0,0,0,0,0,0,0,0,0,0,...,0,0,2,2,0,0,0,0,1,1


In [13]:
# Select features to learn from (genotyping only)
drugs = ['Aripiprazole','Olanzapine','Quetiapine','Risperidone']
X = df_merged.drop(columns=drugs)

# Iterate through drugs as outcome features
for i in drugs:
    Y = df_merged[i]
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
    rf = RandomForestClassifier(n_estimators=100)
    rf.fit(X_train,y_train)
    y_pred = rf.predict(X_test)
    score = accuracy_score(y_test,y_pred)
    print('Outcome: '+str(i))
    print('Classifier: '+'RandomForest')
    print('Accuracy Score: '+str(score))
    print('----------------------------------')

Outcome: Aripiprazole
Classifier: RandomForest
Accuracy Score: 0.4117647058823529
----------------------------------
Outcome: Olanzapine
Classifier: RandomForest
Accuracy Score: 0.7647058823529411
----------------------------------
Outcome: Quetiapine
Classifier: RandomForest
Accuracy Score: 0.5588235294117647
----------------------------------
Outcome: Risperidone
Classifier: RandomForest
Accuracy Score: 1.0
----------------------------------
